# Start

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_ttest

experiment_tables_path = './experiment_tables/'
experiment_tables_dict = None
processed_datasets_df = None

datasets_path = './datasets/'
files = os.listdir(datasets_path)

# Load data

In [2]:
# Here we load the Experiment Table from each quantifier
# The Experiment Table contains every exectuion + results
# from the algorithm.
def load_experiment_tables():
    exp_tables_dict = {key: None for key in ['CC', 'ACC', 'PACC', 'PCC', 'SMM', 'HDy', 'DyS', 'SORD', 'MS', 'MS2', 'MAX', 'X']}
        
    for key in exp_tables_dict.keys():
        if os.path.isfile(experiment_tables_path + 'experiment_table_' + key + '.csv'):
            exp_tables_dict[key] = pd.read_csv(experiment_tables_path + 'experiment_table_' + key + '.csv')
        else:
            exp_tables_dict[key] = pd.DataFrame(columns=['dataset_name', 'alpha', 'sample_size', 'real_p', 'pred_p', 'abs_error', 'run_time'])
    
    return exp_tables_dict

experiment_tables_dict = load_experiment_tables()
df_dict = {key: None for key in list(experiment_tables_dict.keys())}


for key in experiment_tables_dict:
    df_dict[key] = experiment_tables_dict[key].groupby('dataset_name')['abs_error'].aggregate('mean')

meta_features_table = pd.read_csv('./metafeatures/meta-features-table.csv')

algList = []
tableList = []
for counter in df_dict.keys():
    algList.append(counter)

    y = df_dict[counter].values

    X = meta_features_table.values
    np.nan_to_num(X, copy=False)
    
    row, column = np.where(X > np.finfo(np.float32).max)
    for i in range(len(row)):
        X[row[i]][column[i]] = np.finfo(np.float32).max

    tableList.append((X, y))

processed_datasets_df = pd.read_csv('./experiment_tables/processed_datasets.csv')
datasets = processed_datasets_df['dataset'].tolist()

for key in df_dict.keys():
    df = pd.DataFrame({'MAE': df_dict[key].to_list(), 'Dataset': datasets})
    df.to_csv('./experiment_tables/summarized/experiment_table_' + str(key) + '.csv', index = False)
processed_datasets_df = pd.read_csv('./experiment_tables/processed_datasets.csv')

# Train & test recommender with Leave-one-out

This code evaluates the recommender (regressor) with Leave-one-out

In [3]:
# # Regressor ~ Recommender
# rf_reg = RandomForestRegressor()

# instance_len = len(tableList[0][0])
# rf_results = {}

# # tableList contains the meta-table (X, y) from 
# # each quantifier
# #
# # this loop will train and test the RF regressor (recommender)
# # using the meta-table (X, y) with Leave-One-Out
# j = 0
# for (X, y) in tableList:
#   rf_results_list = []
#   algName = algList[j]
#   j += 1

#   # LEAVE-ONE-OUT
#   for i in range(0, len(X)):
#     X_train = np.delete(X, i, 0)
#     y_train = np.delete(y, i, 0)

#     X_test = X[i]
#     X_test = X_test.reshape(1, -1)
#     y_test = y[i]

#     rf_reg.fit(X_train, y_train)
  
#     rf_abs_error = rf_reg.predict(X_test)


#     rf_results_list.append([y_test, rf_abs_error[0]])


#   rf_results[algName] = rf_results_list


# # A results table (named 'recommendation table') is constructed
# # for the regressor (recommender)

# # # # RANDOM FORESTS
# # # #
# data = []
# cols = []
# for key in rf_results:
#   cols.append('abs-error-'+key)
#   cols.append('abs-error-'+key+'-predicted')
# cols.append('abs-error-ideal')
# cols.append('quantifier-ideal')
# cols.append('quantifier-ideal-num')
# cols.append('abs-error-recommended')
# cols.append('quantifier-recommended')
# cols.append('quantifier-recommended-num')
# i = 1
# for key in rf_results:
#     cols.append('rank-' + str(i))
#     i += 1


# i = 0
# for i in range(0, instance_len):
#   abs_error_ideal = 2
#   quantifier_ideal = 'NULL'
#   quantifier_ideal_num = -1
#   abs_error_recommended = 2
#   quantifier_recommended = 'NULL'
#   quantifier_recommended_num = -1
#   row = []
#   algNum = 0
#   rank = {}

#   for a in algList:
#     row.append(rf_results[a][i][0])
#     row.append(rf_results[a][i][1])

#     rank[algNum] = rf_results[a][i][1]

#     if rf_results[a][i][0] < abs_error_ideal:
#       abs_error_ideal = rf_results[a][i][0]
#       quantifier_ideal = a
#       quantifier_ideal_num = algNum

#     if rf_results[a][i][1] < abs_error_recommended:
#       abs_error_recommended = rf_results[a][i][1]
#       quantifier_recommended = a
#       quantifier_recommended_num = algNum

#     algNum += 1
#   rank = sorted(rank.items(), key=lambda item: item[1])

#   row.append(abs_error_ideal)
#   row.append(quantifier_ideal)
#   row.append(quantifier_ideal_num)
#   row.append(abs_error_recommended)
#   row.append(quantifier_recommended)
#   row.append(quantifier_recommended_num)
#   for key in rank:
#     row.append(int(key[0]))

#   data.append(row)

# rf_table = pd.DataFrame(data, columns = cols)

# # # This line saves the results of the leaven-one-out evaluation of the recommender:
# # rf_table.to_csv("./recommendation/recommendation_table_rf.csv", index = False)

# Load Recommendation Table

In [4]:
# We load the Recommendation Table generated in the previous step
rf_table = pd.read_csv("./recommendation/recommendation_table_rf.csv")

# Friedmann Test + Posthoc with Holm Procedure

This code constructs the optimal quantifier sets for each dataset using a non-parametric Friedmann Test + Posthoc with Holm Procedure

In [5]:
# optimal_dict -> for each dataset (key) there is a set of optimal (adequate) quantifiers (for that problem)
# 
# test_dict -> for each dataset (key) there is a dataframe containing the MAE of each quantifier in that problem
# (you can run this code and print 'test_dict' for clarification)
test_dict = {}
optimal_dict = {}
for d in datasets:
    test_dict[d.split('.csv')[0]] = None
    optimal_dict[d.split('.csv')[0]] = None

for dataset in test_dict.keys():
    quantifier_error_dict = {key: [] for key in algList}

    for alg in algList:
        temp_df = experiment_tables_dict[alg].loc[experiment_tables_dict[alg]['dataset_name'] == dataset]
        temp_df = temp_df.groupby('alpha').mean(numeric_only = True)
        temp_df = temp_df.reset_index()
        quantifier_error_dict[alg] = temp_df['abs_error'].tolist()
        
    test_dict[dataset] = pd.DataFrame()
    for key in quantifier_error_dict.keys():
        test_dict[dataset][key] = quantifier_error_dict[key]

# Here we construct the optimal quantifier set for each dataset
#
# The best quantifier is selected (lower error) and then every quantifier
# that is similar is also included (through a Friedmann Test + Posthoc with Holm Procedure)
test_index = 0
for dataset in test_dict.keys():
    groups = []
    for counter in test_dict[dataset].columns.tolist():
        groups.append(test_dict[dataset][counter].tolist())
    res = friedmanchisquare(*groups)
    if res.pvalue < 0.05:
        # HOLM
        best = test_dict[dataset].mean().sort_values().index[0]
        best_index = test_dict[dataset].columns.tolist().index(best)
        opt_set = set()
        opt_set.add(best)

        res = posthoc_ttest(groups, p_adjust='holm')
        
        index = 0
        for element in res[best_index+1]:
            if element >= 0.05:
                opt_set.add(test_dict[dataset].columns.tolist()[index])
            index += 1
        
        optimal_dict[dataset] = opt_set
    else:
        optimal_dict[dataset] = set(algList)

# Topline

This code creates the Topline Experiment Table.

The Topline is the ideal case, which would happen if we choose the best algorithm for every dataset.

In [6]:
topline_experiment_table = pd.DataFrame(columns=['MAE', 'Dataset'])

# Load every dataset (by name)
dt_list = []
for d in datasets:
    dt_list.append(d.split('.csv')[0])

# Get all the Ideal Quantifiers from the Recommendation Table
rows = rf_table['quantifier-ideal'].to_list()
topline_error_list = []

# Include in the Experiment Table the MAE of the 'Topline' Quantifier, which would be the best quantifier
# for that specific dataset
i = 0
for r in rows:
    topline_experiment_table.loc[len(topline_experiment_table)] = [rf_table.loc[i]['abs-error-' + r], dt_list[i]]
    i += 1

# Save the resulting Experiment Table
topline_experiment_table.to_csv('./experiment_tables/summarized/experiment_table_TOPLINE.csv', index=False)

# Construction of the Top-K sets + weights

This code creates two dictionaries that are important for the next steps:

(1) rf_top_dict:

      For each dataset (dictonary key) there is a set of the top-K quantifiers for that dataset (dictonary value).
      This is important to calculate the Hit Rate, by comparing the top-K set with the optimal set.

(2) rf_ensemble_top_dict:

      For each dataset (dictionary key) there is a list containing the Top-K (i) quantifiers and (ii) their respective weights
      This is important for the Top-K Weighted Ensemble Approach, since we need to know the Top-K quantifiers + their respective
      weights. For the Top-K Ensemble Approach (not using weights), we only use the Top-K quantifiers (and then we calculate the
      mean or median of their estimations).

You can run this code and print "rf_top_dict" and "rf_ensemble_top_dict" for better clarification

In [7]:
dt_list = []
for d in datasets:
    dt_list.append(d.split('.csv')[0])

rf_top_dict = {key: None for key in dt_list}

rf_ensemble_top_dict = {}
for key in dt_list:
    rf_ensemble_top_dict[key] = []



# This little variable controls the K in the top-K
top = 5



index = 0
for dataset in rf_top_dict.keys():
    quantifier_set = set()
    ensemble_quantifier_list = []
    ensemble_error_list = []
    for i in range(0, top):
        quantifier_set.add(algList[rf_table.loc[index]['rank-'+str(i+1)]])
        ensemble_quantifier_list.append(algList[rf_table.loc[index]['rank-'+str(i+1)]])
        ensemble_error_list.append(rf_table.loc[index]['abs-error-'+str( algList[rf_table.loc[index]['rank-'+str(i+1)]] )+'-predicted'])
    rf_top_dict[dataset] = quantifier_set
    
    ensemble_numerator_list = []
    denominator = 0
    for error in ensemble_error_list:
        denominator += 1/error
        ensemble_numerator_list.append( 1/error )
    
    ensemble_weight_list = []
    for numerator in ensemble_numerator_list:
        ensemble_weight_list.append( numerator / denominator )

    rf_ensemble_top_dict[dataset].append(ensemble_quantifier_list)
    rf_ensemble_top_dict[dataset].append(ensemble_weight_list)
    
    index += 1

# Top-K Ensemble HitRate

Generate the HitRate for the Top-K ensemble and save to the Recommender Hit Table

In [8]:
rf_top_hit_list = []
index = 0
for dataset in dt_list:
    # if A_recommendation_set INTERSECTION A_optimal_set is not NULL, then a Hit is attributed
    if len(rf_top_dict[dataset].intersection(optimal_dict[dataset])) != 0:
        rf_top_hit_list.append(1)
    else:
        rf_top_hit_list.append(0)
    index += 1

rf_top_hit_rate = np.mean(rf_top_hit_list)


# Save the HitRate to the Recommender Hit Table
temp_table = pd.read_csv('./recommender_hit_table/recommender_hit_rate_table.csv')
temp_table.loc[len(temp_table)] = ['Random Forest+Top'+str(top), rf_top_hit_rate]
# temp_table.to_csv('./recommender_hit_table/recommender_hit_rate_table.csv', index=False)

# Top-K Ensemble

This code constructs the top-K ensemble method

Each recommender predicts a MAE for their specific quantifier

Quantifiers are then ordered by their MAE (crescent order)

The Top-k are selected and are used to give their own estimates of the class prevalence {p_1, ..., p_k}

The "final" prevalence is the mean OR median (we use both approaches) of the k estimations

Then, the absolute error of our approach (mean or median) is calculated -> ABSOLUTE(real_prevalence - estimated_prevalence)

In [9]:
top_k_experiment_table_mean = pd.DataFrame(columns=['MAE', 'Dataset'])
top_k_experiment_table_median = pd.DataFrame(columns=['MAE', 'Dataset'])

dt_list = []
for d in datasets:
    dt_list.append(d.split('.csv')[0])

index = 0
for item in rf_top_dict.items():
    quantifiers_set = item[1]
    
    pred_p_mean = [0] * 20
    temp_pred_p_median = [ [] for _ in range(20) ]
    pred_p_median = []
    real_p = None
    for x in quantifier_set:
        # Get the real_p (prevalence) of the dataset and the predicted_p (prevalence) by the quantifier
        real_p = experiment_tables_dict[x].loc[experiment_tables_dict[x]['dataset_name'] == dt_list[index]].groupby('alpha').mean(numeric_only=True)['real_p'].values.tolist()
        pred_p = experiment_tables_dict[x].loc[experiment_tables_dict[x]['dataset_name'] == dt_list[index]].groupby('alpha').mean(numeric_only=True)['pred_p'].values.tolist()
        for i in range(0, len(real_p)):
            pred_p_mean[i] += pred_p[i]
            temp_pred_p_median[i].append(pred_p[i])
    
    # The "final" prevalence will be the mean or median of each estimated prevalence by each quantifier
    for i in range(0, len(pred_p_mean)):
        pred_p_mean[i] /= len(quantifier_set)
        pred_p_median.append(np.median(temp_pred_p_median[i]))
    
    abs_error_mean_list = []
    abs_error_median_list = []
    for i in range(0, len(pred_p_mean)):
        abs_error_mean_list.append(abs(real_p[i] - pred_p_mean[i]))
        abs_error_median_list.append(abs(real_p[i] - pred_p_median[i]))
    abs_error_mean = np.mean(abs_error_mean_list)
    abs_error_median = np.mean(abs_error_median_list)

    top_k_experiment_table_mean.loc[len(top_k_experiment_table_mean)] = [abs_error_mean, dt_list[index]]
    top_k_experiment_table_median.loc[len(top_k_experiment_table_median)] = [abs_error_median, dt_list[index]]
    # experiment_table.loc[len(experiment_table)] = ['Random Forests+Top'+str(top)+' (MEAN)', dt_list[index], abs_error_mean]
    # experiment_table.loc[len(experiment_table)] = ['Random Forests+Top'+str(top)+' (MEDIAN)', dt_list[index], abs_error_median]

    index += 1

if top == 1:
    top_k_experiment_table_mean.to_csv('./experiment_tables/summarized/experiment_table_TOP'+str(top)+'.csv', index=False)
else:
    top_k_experiment_table_mean.to_csv('./experiment_tables/summarized/experiment_table_TOP'+str(top)+'_MEAN.csv', index=False)
    top_k_experiment_table_median.to_csv('./experiment_tables/summarized/experiment_table_TOP'+str(top)+'_MEDIAN.csv', index=False)

# Top-K Weighted Ensemble

This code implements the weighted top-K ensemble method

Each recommender predicts a MAE for their specific quantifier

Quantifiers are then ordered by their MAE (crescent order)

A weight is attributed to each quantifier based on their MAE. The lower the MAE, the bigger the weight. The first quantifier (Top-1) has the biggest weight, the second (Top-2) has the seccond biggest weight, and so on.

ALL of the weights sum up to one -> weight_quantifier_1 + weight_quantifier_2 + ... + weight_quantifier_k = 1

The Top-k are selected and are used to give it's own estimate of the class prevalence {p_1, ..., p_k}

The "final" prevalence is the sum of the k estimations multiplied by their respective weight:

prevalence_estimation = weight_quantifier_1 * prevalence_quantifier_1 + ... + weight_quantifier_k * prevalence_quantifier_k

Then, the absolute error of our approach is calculated -> ABSOLUTE(real_prevalence - estimated_prevalence)

In [10]:
top_k_weighted_experiment_table = pd.DataFrame(columns=['MAE', 'Dataset'])

dt_list = []
for d in datasets:
    dt_list.append(d.split('.csv')[0])

index = 0
for item in rf_ensemble_top_dict.items():
    quantifier_list = item[1][0]
    weight_list = item[1][1]
    prevalence_list = [0] * 20

    real_p = None
    for x in quantifier_list:
        # Get the real_p (prevalence) of the dataset and the predicted_p (prevalence) by the quantifier
        real_p = experiment_tables_dict[x].loc[experiment_tables_dict[x]['dataset_name'] == dt_list[index]].groupby('alpha').mean(numeric_only=True)['real_p'].values.tolist()
        pred_p = experiment_tables_dict[x].loc[experiment_tables_dict[x]['dataset_name'] == dt_list[index]].groupby('alpha').mean(numeric_only=True)['pred_p'].values.tolist()
        
        # The "final" prevalence will be the prevalences of the K quantifiers adjusted by their weights.
        # Note that pred_p has 20 prevalences, since we test a quantifier on a dataset 20 times (varying the class distribution).
        for i in range(0, len(pred_p)):
            prevalence_list[i] += pred_p[i] * weight_list[quantifier_list.index(x)]
    
    ensemble_abs_error_list = []
    for i in range(0, len(prevalence_list)):
        ensemble_abs_error_list.append( abs( real_p[i] - prevalence_list[i] ) )

    ensemble_abs_error = np.mean(ensemble_abs_error_list)
    
    top_k_weighted_experiment_table.loc[len(top_k_weighted_experiment_table)] = [ensemble_abs_error, dt_list[index]]
    # experiment_table.loc[len(experiment_table)] = ['Random Forests+Top'+str(top)+'+Weighted (ENSEMBLE)', dt_list[index], ensemble_abs_error]
    index += 1

if top != 1: 
    top_k_weighted_experiment_table.to_csv('./experiment_tables/summarized/experiment_table_TOP'+str(top)+'_WEIGHTED.csv', index=False)

# Unify all the Experiment Tables

In the previous steps, we created Experiment Tables for each quantifier. The experiment tables for our Top-K approach were created here, while the experiment tables for the other quantifiers were created in the APP function and were summarized here (refer to 'summarized' folder).

Now we are going to unify all these experiment tables into a single Experiment Table (even more summarized) consisting of a DataFrame containing (1) the method (algorithm), (2) the Dataset name and (3) MAE.

This is just to facilitate our analysis and to condense all the results to a single file.

In [11]:
# The 'even more summarized' Experiment Table containts only the Method, Dataset and MAE
even_more_summarized_experiment_table = pd.DataFrame(columns=['Method', 'Dataset', 'MAE'])

dt_list = []
for d in datasets:
    dt_list.append(d.split('.csv')[0])

for filename in os.listdir('./experiment_tables/summarized/'):
    # We load each experiment table
    df = pd.read_csv('./experiment_tables/summarized/'+filename)
    string = filename.split('_')
    method_name = string[2].split('.')[0]
    
    if 'TOP' in method_name and method_name != 'TOPLINE':
        if '1' not in method_name:
            method_name = method_name + '+' + string[3].split('.')[0]

    # And then we include each row in the 'even more summarized' one
    index = 0
    for row in df.iterrows():
        even_more_summarized_experiment_table.loc[len(even_more_summarized_experiment_table)] = [method_name, dt_list[index], row[1]['MAE']]
        index += 1

# Save the resulting 'even more summarized' experiment table
even_more_summarized_experiment_table.to_csv('./experiment_tables/even_more_summarized/experiment_table.csv', index=False)

# TEST

In [24]:
experiment_table = pd.read_csv('./experiment_tables/even_more_summarized/experiment_table_2.csv')
old_experiment_table = pd.read_csv('./experiment_tables/even_more_summarized/experiment_table_3.csv')

In [25]:
# old_experiment_table['Method'].replace({'Random Forests+Top1' : 'TOP1',
#                                         'Random Forests+Top3 (MEAN)' : 'TOP3+MEAN',
#                                         'Random Forests+Top3 (MEDIAN)' : 'TOP3+MEDIAN',
#                                         'Random Forests+Top3+Weighted (ENSEMBLE)' : 'TOP3+WEIGHTED',
#                                         'Random Forests+Top5 (MEAN)' : 'TOP5+MEAN',
#                                         'Random Forests+TOP5 (MEDIAN)' : 'TOP5+MEDIAN',
#                                         'Random Forests+Top5+Weighted (ENSEMBLE)' : 'TOP5+WEIGHTED',
#                                         'Topline' : 'TOPLINE'}, inplace=True)

In [26]:
for quantifier in experiment_table['Method'].unique().tolist():
    print(quantifier + ' : ' + str(set(experiment_table.query("Method in ['"+quantifier+"']")['MAE'].tolist()) == set(old_experiment_table.query("Method in ['"+quantifier+"']")['MAE'].tolist())))

ACC : True
CC : True
DyS : True
HDy : True
MAX : True
MS : True
MS2 : True
PACC : True
PCC : True
SMM : True
SORD : True
TOP1 : False
TOP3+MEAN : False
TOP3+MEDIAN : False
TOP3+WEIGHTED : False
TOP5+MEAN : False
TOP5+MEDIAN : False
TOP5+WEIGHTED : False
TOPLINE : True
X : True


In [15]:
MAE_1 = experiment_table.query("Method in ['TOP1']")['MAE'].tolist()
MAE_2 = old_experiment_table.query("Method in ['TOP1']")['MAE'].tolist()

In [16]:
MAE_1

[0.04085,
 0.1025,
 0.0471,
 0.5,
 0.0271999999999999,
 0.1624499999999999,
 0.43655,
 0.337,
 0.0227499999999999,
 0.263,
 0.4755999999999999,
 0.32255,
 0.1075499999999999,
 0.4544,
 0.26505,
 0.0,
 0.1331999999999999,
 0.0191999999999999,
 0.0066,
 0.27835,
 0.0947,
 0.0031499999999999,
 0.0510499999999999,
 0.08465,
 0.0,
 0.317,
 0.5,
 0.0319,
 0.0202499999999999,
 0.10305,
 0.0773499999999999,
 0.1075,
 0.0561,
 0.0200999999999999,
 0.5,
 0.0376,
 0.0224999999999999,
 0.0274,
 0.01705,
 0.04,
 0.0104,
 0.0069499999999999,
 0.0009,
 0.281,
 0.0024499999999999,
 0.01765,
 0.0045999999999999,
 0.0251,
 0.482,
 0.0252499999999999,
 0.27265,
 0.00905,
 0.0232499999999999,
 0.02235,
 0.0905499999999999,
 0.07235,
 0.0286,
 0.0774999999999999,
 0.5,
 0.5,
 0.0315,
 0.0894999999999999,
 0.4005500000000001,
 0.1421999999999999,
 0.0843,
 0.1036,
 0.0452999999999999,
 0.0638499999999999,
 0.0238999999999999,
 0.0030999999999999,
 0.0264,
 0.0064,
 0.0135999999999999,
 0.0479,
 0.0488999999

In [17]:
i = 0
for i in range(0, len(MAE_1)):
    print(str(MAE_1[i]) + '\t\t|\t\t' + str(MAE_2[i]) + ' : ' + str(abs(MAE_1[i] - MAE_2[i])))


0.04085		|		0.0397 : 0.0011499999999999982
0.1025		|		0.1409999999999999 : 0.03849999999999991
0.0471		|		0.0193499999999999 : 0.027750000000000104
0.5		|		0.5 : 0.0
0.0271999999999999		|		0.0383999999999999 : 0.011199999999999998
0.1624499999999999		|		0.1554499999999999 : 0.007000000000000006
0.43655		|		0.1617499999999999 : 0.2748000000000001
0.337		|		0.5 : 0.16299999999999998
0.0227499999999999		|		0.0472 : 0.0244500000000001
0.263		|		0.5 : 0.237
0.4755999999999999		|		0.49895 : 0.023350000000000093
0.32255		|		0.23645 : 0.08610000000000001
0.1075499999999999		|		0.1088 : 0.0012500000000000983
0.4544		|		0.5 : 0.045599999999999974
0.26505		|		0.28385 : 0.018799999999999983
0.0		|		0.0 : 0.0
0.1331999999999999		|		0.08255 : 0.0506499999999999
0.0191999999999999		|		0.0265499999999999 : 0.007349999999999999
0.0066		|		0.00335 : 0.00325
0.27835		|		0.1795499999999999 : 0.09880000000000008
0.0947		|		0.3006 : 0.20589999999999997
0.0031499999999999		|		0.0033499999999999 : 0.000200000

In [18]:
experiment_table['Method'].unique().tolist()

['ACC',
 'CC',
 'DyS',
 'HDy',
 'MAX',
 'MS',
 'MS2',
 'PACC',
 'PCC',
 'SMM',
 'SORD',
 'TOP1',
 'TOP3+MEAN',
 'TOP3+MEDIAN',
 'TOP3+WEIGHTED',
 'TOP5+MEAN',
 'TOP5+MEDIAN',
 'TOP5+WEIGHTED',
 'TOPLINE',
 'X']

In [19]:
old_experiment_table['Method'].replace({'Random Forests+Top1' : 'TOP1',
                                        'Random Forests+Top3 (MEAN)' : 'TOP3+MEAN',
                                        'Random Forests+Top3 (MEDIAN)' : 'TOP3+MEDIAN',
                                        'Random Forests+Top3+Weighted (ENSEMBLE)' : 'TOP3+WEIGHTED',
                                        'Random Forests+Top5 (MEAN)' : 'TOP5+MEAN',
                                        'Random Forests+TOP5 (MEDIAN)' : 'TOP5+MEDIAN',
                                        'Random Forests+Top5+Weighted (ENSEMBLE)' : 'TOP5+WEIGHTED'}, inplace=True)

In [20]:
old_experiment_table['Method'].unique()

array(['ACC', 'CC', 'DyS', 'HDy', 'MAX', 'MS', 'MS2', 'PACC', 'PCC',
       'SMM', 'SORD', 'TOP1', 'TOP3+MEAN', 'TOP3+MEDIAN', 'TOP3+WEIGHTED',
       'TOP5+MEAN', 'TOP5+MEDIAN', 'TOP5+WEIGHTED', 'TOPLINE', 'X'],
      dtype=object)

In [21]:
old_experiment_table.query("Method in ['CC']")['MAE'].tolist()

[0.1832,
 0.19905,
 0.28845,
 0.5,
 0.27,
 0.38815,
 0.33715,
 0.5,
 0.12955,
 0.5,
 0.4993999999999999,
 0.43395,
 0.1975,
 0.5,
 0.3929999999999999,
 0.0,
 0.37205,
 0.0317,
 0.00885,
 0.3398,
 0.42095,
 0.00955,
 0.4485,
 0.3601,
 0.0,
 0.5,
 0.5,
 0.1902499999999999,
 0.22625,
 0.0762,
 0.215,
 0.4203499999999999,
 0.22795,
 0.24815,
 0.5,
 0.08525,
 0.0233999999999999,
 0.14825,
 0.13335,
 0.22665,
 0.00975,
 0.02005,
 0.0,
 0.5,
 0.0151,
 0.1303,
 0.01665,
 0.0538999999999999,
 0.5,
 0.3148999999999999,
 0.2978,
 0.00995,
 0.16845,
 0.2118,
 0.217,
 0.2162,
 0.20975,
 0.34765,
 0.5,
 0.5,
 0.4853499999999999,
 0.47485,
 0.5,
 0.25215,
 0.1448,
 0.2146,
 0.3625,
 0.25295,
 0.02845,
 0.02135,
 0.02535,
 0.0086,
 0.0204,
 0.03705,
 0.22735,
 0.05325,
 0.0786,
 0.02075,
 0.1417,
 0.22195,
 0.02845,
 0.5,
 0.2161,
 0.1877499999999999,
 0.5,
 0.02555,
 0.32205,
 0.0103,
 0.0592,
 0.0,
 0.173,
 0.1582,
 0.16365,
 0.2196999999999999,
 0.36655,
 0.011,
 0.0917999999999999,
 0.38775,
 0.04